## Expedited patterns processing
This notebook reads in the patterns data and:
    1. Filters down to Philadelphia zipcodes based on a (currently hard-coded) list.
        a. I have not yet run this using this list. Previously it filtered down by looking at the state and city columns in the data
    3. Concatenates the files
    4. Writes the result to philly_patterns.csv in the processed data folder.
    
It takes a long time to run and the resulting dataset is very large so it is worth thinking about ways to cut down the data.

In [1]:
import pandas as pd
from safegraph_py_functions import safegraph_py_functions as sgpy
import os
from dotenv import load_dotenv, find_dotenv

In [5]:
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)
root_dir = os.environ.get("ROOT_DIR")
raw_data_dir = os.path.join(root_dir,'data/raw')

In [6]:
# local directory where we want to put all the data
patterns_path = os.path.join(raw_data_dir,'monthly-patterns')
# print(local)
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(patterns_path):
    for file in f:
        if file.endswith('.csv.gz') and 'patterns-part' in file:
            files.append(os.path.join(patterns_path, r, file))
#print(files)

In [ ]:
# This list was taken from https://www.city-data.com/zipmaps/Philadelphia-Pennsylvania.html
philly_zips = pd.Series(['19102', '19103', '19104', '19106', '19107', '19109', '19111', '19112', '19114', '19115', 
               '19116', '19118', '19119', '19120', '19121', '19122', '19123', '19124', '19125', '19126', 
               '19127', '19128', '19129', '19130', '19131', '19132', '19133', '19134', '19135', '19136', 
               '19137', '19138', '19139', '19140', '19141', '19142', '19143', '19144', '19145', '19146', 
               '19147', '19148', '19149', '19150', '19151', '19152', '19153', '19154'], name = 'postal_code')

philly_zips = philly_zips.to_frame()

In [12]:
keep_cols = ['safegraph_place_id', 'location_name','street_address','city','region','postal_code',
             'date_range_start', 'date_range_end', 'raw_visit_counts','raw_visitor_counts','visits_by_day',
            'poi_cbg']

def filter_and_explode(df):
    df = df[keep_cols]
    # zip codes are read as integers rather than strings so we add leading zeros.
    # this is not strictly necessary since Philadelphia zipcodes don't have leading zeros.
    df['postal_code'] = df['postal_code'].apply(lambda x: ('00000' + str(x))[-5:])
    df = philly_zips.merge(df)
    # The visits_by_day column contains a list of integers. 
    # This explodes that list so we get one row per day.
    df = sgpy.explode_json_array(
        df, array_column ='visits_by_day', value_col_name='day_visit_counts', 
        place_key='safegraph_place_id', file_key='date_range_start', array_sequence='day', 
        keep_index=False, zero_index=False)
    df['date_range_start'] = pd.to_datetime(df['date_range_start'])
    temp = df['day'].apply(lambda x: pd.Timedelta(x-1, unit='D'))
    df['date'] = df['date_range_start'] + temp
    return df

In [14]:
philly_patterns = [filter_and_explode(pd.read_csv(file)) for file in files]

In [18]:
philly_patterns_df = pd.concat(philly_patterns)

In [15]:
processed_data_dir = os.path.join(root_dir,'data/processed')

In [19]:
philly_patterns_df.to_csv(os.path.join(processed_data_dir,'philly_patterns.csv'), index = False)